### Preparación datos modelo

Preparar los datos para los modelos de inferencia.

Necesita mergear:
* Archivo con los tweets y los stats para cada uno de ellos (solo nos interesan lo de engagement).
    * Dejar la fecha solo como año/mes/dia.
    * Eliminar RTs, el resto nos los podemos quedar.
* Archivo con las dimensiones para cada uno.
    * Sería una columna por dimensión. Dimensión original si se está usando el modelo de Aiello.

Hacemos primero el filtrado de los municipios que nos interesan (de acuerdo a la cantidad de tweets en cada período).

* Normaliza con percentil y de acuerdo al ``word_length``.
* Summariza de acuerdo a tweet/user_id y luego por distintos períodos temporales none/day/week/month.

In [ ]:
import pandas as pd
from collections import deque
from tqdm.notebook import tqdm

In [ ]:
# las fechas y el df_stats para filtrar los que nos interesan
from datetime import datetime

data_dir = './df_merged/'
inic_no_covid = datetime(2019,1,1)
no_covid = datetime(2020,1,1)
pre_covid = datetime(2020,3,1)
post_covid = datetime(2022,1,1)

df_stats = pd.read_pickle(data_dir + 'df_date_stats.pickle')
df_stats = df_stats[(df_stats['no_covid'] > 0) & (df_stats['pre_covid'] > 0) & (df_stats['during_covid'] > 0) & (df_stats['post_covid'] > 0)]
len(df_stats)

In [ ]:
df_tweets = pd.read_pickle(data_dir + 'df_tweets_db.pickle')
if df_tweets['created_at'].values[0] is datetime:
    df_tweets['created_at'] = df_tweets['created_at'].dt.date
df_tweets = df_tweets[df_tweets['retweeted_id'] == -1]
df_tweets = df_tweets[df_tweets['user_id'].isin(df_stats.index)]
len(df_tweets)

In [ ]:
# definimos las dimensiones para trabajar
ten_dims = ['knowledge','power','status','trust','support','romance','similarity','identity','fun','conflict']

In [ ]:
# mergeamos la info de las cuentas de usuario (variables de control según los papers)
df_user_stats = pd.read_pickle(data_dir + 'df_user_current_stats.pickle')
df_user_stats

df_tweets = df_tweets.reset_index().merge(df_user_stats,left_on='user_id',right_on='user_id').set_index('tweet_id')
df_tweets

In [ ]:
# agregamos columnas que podrían ser de control para los tweets
df_tweets['hashtag_count'] = df_tweets['hashtags'].map(len)
df_tweets['media_count'] = df_tweets['media'].map(len)
df_tweets['mentions_count'] = df_tweets['mentions'].map(len)
df_tweets['url_count'] = df_tweets['url'].map(len)

In [ ]:
# nos quedamos solo con las columnas que son de interés
df_tweets_reduced = df_tweets[['created_at','user_id','favorite_count','retweet_count','reply_count','word_length','sentence_length','hashtag_count','media_count','mentions_count','url_count','followers','followees','listed','tweets']]
df_tweets_reduced

In [ ]:
# add column tweets_before -- como sacamos la hora de los tweets, vamos a contar hasta el día anterior de cada uno
# esto hace que tweets del mismo día tengan la misma cantidad de tweets previos de ese usuario
import os

if os.path.exists(data_dir + 'df_tweets_reduced.pickle'):
    dd = pd.read_pickle(data_dir + 'df_tweets_reduced.pickle')
    df_tweets_reduced = df_tweets_reduced.merge(dd[['tweets_before']],left_index=True,right_index=True)
else:
    dd = deque()
    user_dates = {}
    for i in tqdm(range(0,len(df_tweets_reduced))):
        uu = df_tweets_reduced['user_id'].values[i]
        if uu not in user_dates: # al menos que no tenga que recalcular los tweets de un usuario en particular, siendo que hay solo 80 usuarios 
            user_dates[uu] = list(df_tweets_reduced[df_tweets_reduced['user_id'] == uu]['created_at'].values)

        d = df_tweets_reduced['created_at'].values[i]
        dd.append(sum(1 for x in user_dates[uu] if x < d))

    df_tweets_reduced['tweets_before'] = dd

    df_tweets_reduced.to_pickle(data_dir + 'df_tweets_reduced.pickle')

df_tweets_reduced.columns

In [ ]:
file_name = 'df_tweets_social_dimensions_empath_median-no-zeros.pickle' # TODO: Acá se cambia el archivo
file_name = 'df_tweets_social_dimensions_model_simplified.pickle'

df_dimensions = pd.read_pickle(data_dir + file_name) 

df_merged = pd.merge(df_tweets_reduced, df_dimensions,left_index=True,right_index=True)
df_merged

In [ ]:
df_merged.columns

In [ ]:
# en cualquiera de los casos, se hace el promedio o la media para agrupar
# primero se hace la división en tiempo y luego en lo que sea

# time_unit: tweet, day, week, month, phase
# agg_muni: None, user_id
def get_representation(df,time_unit=None,agg_muno='tweet',agg_func='median'): 
    
    df_agg = None
    if agg_muno == 'tweet': # la unidad base va a ser el tweet
        df_agg = pd.DataFrame(df.to_dict())
#         df_agg = df_agg.drop(columns=['user_id'])
    else:
        df_agg = df.groupby(by=['user_id','created_at']).agg(agg_func).reset_index().set_index('user_id')
        
    if time_unit is None: # row: tweet, columns: features
#         if 'created_at' in df_agg.columns:
#             df_agg = df_agg.drop(columns=['created_at'])
        return df_agg
    elif time_unit == 'day': # row: day, columns: features
        df_agg = df_agg.groupby(by='created_at').agg(agg_func) 
        return df_agg
    elif time_unit == 'week': # row: week, columns: features
        df_agg['created_at'] = pd.to_datetime(df_agg['created_at']).dt.to_period('W')
        df_agg = df_agg.groupby(by='created_at').agg(agg_func) 
        return df_agg
    elif time_unit == 'month': #row: month, columns: features
        df_agg['created_at'] = pd.to_datetime(df_agg['created_at']).dt.to_period('M')
        df_agg = df_agg.groupby(by='created_at').agg(agg_func) 
        return df_agg

In [ ]:
# este se puede usar todo junto, o dividido por los tramos y a cada uno se le aplica la transformación 
# y se lo guarda listo para ser usado, presumiblemente en otra notebook
# get_representation(df_merged,agg_muno='user_id',time_unit='week')

In [ ]:
# distribución para determinar si tiene o no tiene el trait
# se puede aplicar antes o después de obtener la representación, con lo que pueden ser valores puros o valores promediados

def normalize_scores(df,columns, score_norm=None, perc=0.85, words_norm=None):
    df_norm = pd.DataFrame(df.to_dict())
    
    if score_norm == 'perc': # todo aquello que tenga un valor por encima del perc pasado por parámetro es 1, el resto 0
        df_norm[columns] = (df_norm[columns] > df_norm[columns].quantile(perc)).astype('int')
    
    if words_norm == 'length_discount': # este es columna a columna porque los valores incluidos en la standararización varían
        for c in columns: # por cada una de las columnas
            mean = df_norm[df_norm[c] > 0]['word_length'].mean()
            std = df_norm[df_norm[c] > 0]['word_length'].std()
            vv = deque()
            for i in range(0, len(df_norm)):
                ss = df_norm[c].values[i]
                if ss == 0:
                    vv.append(0)
                else:
                    zscore = (df_norm['word_length'].values[i] - mean) / std
                    if zscore > 0:
                        vv.append(ss * (1 / (1 + zscore))) # si está normalizado, multiplica por 1, sino por el score real
                    else:
                        vv.append(ss * (2 - (1 / (1 - zscore))))
                    
            df_norm[c] = vv
    
    return df_norm

In [ ]:
df_norm = normalize_scores(df_merged,ten_dims,score_norm='perc')
df_norm

In [ ]:
ww = 'word_length'
# ww = 'sentence_length'
for x in ten_dims:
    print(x,' yes -- ',df_norm[df_norm[x] > 0][[ww]].describe().to_dict())
    print(x,' no -- ',df_norm[df_norm[x] == 0][[ww]].describe().to_dict())
    print('--------')

#### Generación de los archivos de entrada del modelo

El input base para todo es ``df_merged``.

Generar archivos base:
•	Con o sin normalización
•	Con o sin length standarizacion
•	Todos juntos o divididos por fecha (lo de normalizar debería hacerse después de esto)
•	Con diferentes granularidades, tweet/user, None/day/week/month

1.	Todo junto o por grupo de fechas.
2.	Normalización + standarización
3.	Granularidad

No deberían ser muchos archivos los que se generen, porque tampoco queremos probar todo YA. Lo de agrupar por tweet/user tiene sentido que se haga al final.
Lo malo de esto es que vamos a tener muchos promedios/promedios.


In [ ]:
def get_variants(df,prefix,file_name):
    agg = ['tweet','user_id']
    time = [None,'day','week','month']

    for aa in agg:
        for tt in time:
            print(prefix+aa+'_'+str(tt)+'__' + file_name)
            df_summ = get_representation(df,agg_muno=aa,time_unit=tt)
            print(len(df_summ))
            df_summ.to_pickle(prefix+aa+'_'+str(tt)+'__' + file_name)

In [ ]:
# sin separar por fecha -- normaliza todo junto y luego hace las agregaciones correspondientes
df_norm = normalize_scores(df_merged,ten_dims,score_norm='perc')
get_variants(df_norm,'df_merged__all__normalized_perc_85__',file_name)

In [ ]:
df_norm = normalize_scores(df_merged,ten_dims,score_norm=None)
get_variants(df_norm,'df_merged__all__',file_name)

In [ ]:
# sin separar por fecha -- hace las dos normalizaciones todo junto y luego hace las agregaciones correspondientes
df_norm = normalize_scores(df_merged,ten_dims,score_norm='perc',words_norm='length_discount')
get_variants(df_norm,'df_merged__all__normalized_perc_85_word_length__',file_name)

In [ ]:
# dividimos en los grupos por fecha, por cada uno de los grupos, hacemos lo de antes
# estos se podrían separar de los otros y que luego solo haya que hacer el merge que corresponda

df_merged_no_covid = df_merged[(df_merged['created_at'] <= no_covid.date())]
print(len(df_merged_no_covid))

df_merged_pre_covid = df_merged[(df_merged['created_at'] > no_covid.date()) & (df_merged['created_at'] <= pre_covid.date())]
print(len(df_merged_pre_covid))

df_merged_during_covid = df_merged[(df_merged['created_at'] > pre_covid.date()) & (df_merged['created_at'] <= post_covid.date())]
print(len(df_merged_during_covid))

df_merged_post_covid = df_merged[(df_merged['created_at'] > post_covid.date())]
print(len(df_merged_post_covid))

In [ ]:
# df_norm = normalize_scores(df_merged_no_covid,ten_dims,score_norm=None)
# get_variants(df_norm,'df_merged__no_covid__',file_name)

# df_norm = normalize_scores(df_merged_no_covid,ten_dims,score_norm='perc')
# get_variants(df_norm,'df_merged__no_covid__normalized_perc_85__',file_name)

# df_norm = normalize_scores(df_merged_no_covid,ten_dims,score_norm='perc',words_norm='length_discount')
# get_variants(df_norm,'df_merged__no_covid__normalized_perc_85_word_length__',file_name)

df_norm = normalize_scores(df_merged_pre_covid,ten_dims,score_norm=None)
print(len(df_norm))
get_variants(df_norm,'df_merged__pre_covid__',file_name)

# df_norm = normalize_scores(df_merged_pre_covid,ten_dims,score_norm='perc')
# get_variants(df_norm,'df_merged__pre_covid__normalized_perc_85__',file_name)

# df_norm = normalize_scores(df_merged_pre_covid,ten_dims,score_norm='perc',words_norm='length_discount')
# get_variants(df_norm,'df_merged__pre_covid__normalized_perc_85_word_length__',file_name)

df_norm = normalize_scores(df_merged_during_covid,ten_dims,score_norm=None)
print(len(df_norm))
get_variants(df_norm,'df_merged__during_covid__',file_name)

# df_norm = normalize_scores(df_merged_during_covid,ten_dims,score_norm='perc')
# get_variants(df_norm,'df_merged__during_covid__normalized_perc_85__',file_name)

# df_norm = normalize_scores(df_merged_during_covid,ten_dims,score_norm='perc',words_norm='length_discount')
# get_variants(df_norm,'df_merged__during_covid__normalized_perc_85_word_length__',file_name)

df_norm = normalize_scores(df_merged_post_covid,ten_dims,score_norm=None)
print(len(df_norm))
get_variants(df_norm,'df_merged__post_covid__',file_name)

# df_norm = normalize_scores(df_merged_post_covid,ten_dims,score_norm='perc')
# get_variants(df_norm,'df_merged__post_covid__normalized_perc_85__',file_name)

# df_norm = normalize_scores(df_merged_post_covid,ten_dims,score_norm='perc',words_norm='length_discount')
# get_variants(df_norm,'df_merged__post_covid__normalized_perc_85_word_length__',file_name)

In [ ]:
# df_merged_no_covid_2019 = df_merged[(df_merged['created_at'] >= inic_no_covid.date()) & (df_merged['created_at'] <= no_covid.date())]
# print(len(df_merged_no_covid_2019))
# get_variants(df_merged_no_covid_2019,'df_merged__no_covid_2019__',file_name)

df_norm = normalize_scores(df_merged_no_covid_2019,ten_dims,score_norm='perc')
print(len(df_norm))
get_variants(df_norm,'df_merged__no_covid_2019__normalized_perc_85__',file_name)

In [ ]:
df_merged_no_covid_2019